In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math

In [2]:

##? 1st - left, 2nd - right, 3rd - up, 4th - down
##? (x, y) - order

npType = np.int16

SquareVersors = np.array([[-1, 0], [1, 0], [0, 1], [0, -1]], dtype=npType)

##! F points

arrFC = np.array([1, 0, 0, 0], dtype=npType)
arrFB = np.array([0, 1, 0, 0], dtype=npType)
arrFE = np.array([0, 0, 0, 1], dtype=npType)
arrFD = np.array([0, 0, 1, 0], dtype=npType)


##! B points

arrBC = np.array([1, 0, 1, 1], dtype=npType)
arrBB = np.array([0, 1, 1, 1], dtype=npType)
arrBE = np.array([1, 1, 0, 1], dtype=npType)
arrBD = np.array([1, 1, 1, 0], dtype=npType)


##! V points

arrVC = np.array([1, 0, 0, 1], dtype=npType)
arrVB = np.array([0, 1, 1, 0], dtype=npType)
arrVE = np.array([0, 1, 0, 1], dtype=npType)
arrVD = np.array([1, 0, 1, 0], dtype=npType)


###! L points

arrLH = np.array([0, 0, 1, 1], dtype=npType)
arrLV = np.array([1, 1, 0, 0], dtype=npType)


In [3]:
dictStr2Arr = { 
    'FC': arrFC, 'FB': arrFB, 'FE': arrFE, 'FD': arrFD,
    'BC': arrBC, 'BB': arrBB, 'BE': arrBE, 'BD': arrBD,
    'VC': arrVC, 'VB': arrVB, 'VE': arrVE, 'VD': arrVD,
    'LH': arrLH, 'LV': arrLV
}

lookUpFunc = lambda x: dictStr2Arr.get(x, x)  # Default to x if not found in dict

In [4]:

##! improve:vec multiplication outside the loop
##! Auxiliar functions to get the points of the square

def getPointsSquare(x, y, arr):
    pos0 = np.array([2*x+1, 2*y+1], dtype=npType)
    vec = arr.reshape(-1, 1)*SquareVersors
    
    mask = np.all(vec == 0, axis=1)
    result = np.full_like(vec, -1)
    result[~mask] = pos0 + vec[~mask]

    return result


vecTot = np.array([1, 1, 1, 1], dtype=npType)

def getAllPointsSquare(x, y,):
    pos0 = np.array([2*x+1, 2*y+1], dtype=npType)
    vec = vecTot.reshape(-1, 1)*SquareVersors
    
    result = vec + pos0
    return result

In [5]:
def goalTest(tableInit):
    
    tableArr = np.array([[lookUpFunc(x) for x in row] for row in tableInit], dtype=npType)
    
    if tableArr.shape[0] != tableArr.shape[1]:
        return False
    
    mSize = tableArr.shape[0]
    
    dataPointsGrid = np.array([[getPointsSquare(x, y, tableArr[x,y]) for y in range(mSize)] for x in range(mSize)], dtype=npType)
    
    dotsX2 = np.zeros((mSize-1, mSize, 2, 2), dtype=npType) ##! slices along x constant
    dotsY2 = np.zeros((mSize, mSize-1, 2, 2), dtype=npType) ##! slices along y constant
    
    dotsX2[:, :, 0, :] = dataPointsGrid[:-1, :, 1]
    dotsX2[:, :, 1, :] = dataPointsGrid[1:, :, 0]

    dotsY2[:, :, 0, :] = dataPointsGrid[:, :-1, 2]
    dotsY2[:, :, 1, :] = dataPointsGrid[:, 1:, 3]
    
    matLogicalX = (dotsX2[:, :, 0, :] == dotsX2[:, :, 1, :]) & (np.any(dotsX2[:, :, 1, :] > -1)) & (np.any(dotsX2[:, :, 0, :] > -1))
    matLogicalY = (dotsY2[:, :, 0, :] == dotsY2[:, :, 1, :]) & (np.any(dotsX2[:, :, 1, :] > -1)) & (np.any(dotsX2[:, :, 1, :] > -1))
    
    resLogicalX = np.all(matLogicalX, axis=2)
    resLogicalY = np.all(matLogicalY, axis=2)
    
    return resLogicalX, resLogicalY
    # return np.all(resLogicalX) and np.all(resLogicalY)

In [6]:

def deterministicInference(tableInit):

    tableArr = np.array([[lookUpFunc(x) for x in row] for row in tableInit], dtype=npType)

    if tableArr.shape[0] != tableArr.shape[1]:
        return False
    mSize = tableArr.shape[0]
    
    
    dataPointsGrid = np.array([[getPointsSquare(x, y, tableArr[x,y]) for y in range(mSize)] for x in range(mSize)], dtype=npType)
    
    setF = np.array(['FC', 'FB', 'FE', 'FD'])
    setB = np.array(['BC', 'BB', 'BE', 'BD'])
    setV = np.array(['VC', 'VB', 'VE', 'VD'])
    setL = np.array(['LH', 'LV'])


    maskBorder = np.full_like(tableInit, False)
    maskBorder[0, :] = True
    maskBorder[-1, :] = True
    maskBorder[:, 0] = True
    maskBorder[:, -1] = True
    
    ##! putting the deterministic pices in the proper place
    if mSize > 2:
        
        ##! Include the V deterministic pices in the corners
        
        ##! Obtain the position of the external B and L pieces
        idxMatB = np.isin(tableInit, setB)
        idxMatL = np.isin(tableInit, setL)
        
        idxMatBL = np.logical_or(idxMatB, idxMatL)
        
        matDetermWrong = np.logical_and(np.any(np.logical_or(dataPointsGrid == 0,dataPointsGrid == 2*mSize), axis=(2,3)), idxMatBL)
        matDetermRight = np.logical_and(np.logical_and(idxMatBL, np.logical_not(matDetermWrong)), maskBorder)
        
        pointsDeterm = np.where(np.logical_and(idxMatBL, matDetermWrong))
        
        idxDeterm = np.array(list(zip(*pointsDeterm)))
        
        idGridDeterm = np.array([tableInit[x, y] for x, y in zip(*pointsDeterm)])
        
        ##! Put the deterministic pieces in the proper place
        for i in range(idxDeterm.shape[0]):
            x, y = idxDeterm[i]
            idPoint = tableInit[x, y]
            
            if idPoint in setB:
                
                ##? future improvement: remove the wrong pieces from the set at trying to put them in the right place
                gridPointsSet = np.array([getPointsSquare(x, y, lookUpFunc(piece)) for piece in setB], dtype=npType)
                checkBoundaries = np.any(np.logical_or(gridPointsSet == 0, gridPointsSet == 2*mSize), axis=(1,2))
                
                idxCheckBoundaries = np.where(~checkBoundaries)[0]
                
                if len(idxCheckBoundaries) > 1:
                    print("error: more than one possible solution")
                else:
                    tableInit[x, y] = setB[idxCheckBoundaries[0]]
                    tableArr[x, y] = lookUpFunc(setB[idxCheckBoundaries[0]])
                
            elif idPoint in setL:
                
                gridPointsSet = np.array([getPointsSquare(x, y, lookUpFunc(piece)) for piece in setL], dtype=npType)
                checkBoundaries = np.any(np.logical_or(gridPointsSet == 0, gridPointsSet == 2*mSize), axis=(1,2))
                
                idxCheckBoundaries = np.where(~checkBoundaries)[0]
                
                if len(idxCheckBoundaries) > 1:
                    print("error: more than one possible solution")
                else:
                    tableInit[x,y] = setL[idxCheckBoundaries[0]]
                    tableArr[x, y] = lookUpFunc(setL[idxCheckBoundaries[0]])
    else:
        matDetermWrong = np.full_like(tableInit, False)
        matDetermRight = np.full_like(tableInit, False)
    
    
    if mSize < 4:
        ##! include V pieces in the deterministic pieces of matrices with size n x n < 4 x 4
        
        idxMatV = np.isin(tableInit, setV)
        
        matDetermWrongV = np.logical_and(np.any(np.logical_or(dataPointsGrid == 0,dataPointsGrid == 2*mSize), axis=(2,3)), idxMatV)
        matDetermRightV = np.logical_and(np.logical_and(idxMatV, np.logical_not(matDetermWrongV)), maskBorder)
        
        pointsDetermV = np.where(np.logical_and(idxMatV, matDetermWrongV))
        
        idxDetermV = np.array(list(zip(*pointsDetermV)))
        idGridDetermV = np.array([tableInit[x, y] for x, y in zip(*idxDetermV)])
        
        for i in range(idxDetermV.shape[0]):
            x, y = idxDetermV[i]
        
            gridPointsSet = np.array([getPointsSquare(x, y, lookUpFunc(piece)) for piece in setV], dtype=npType)
            checkBoundaries = np.any(np.logical_or(gridPointsSet == 0, gridPointsSet == 2*mSize), axis=(1,2))
                
            idxCheckBoundaries = np.where(~checkBoundaries)[0]
                
            if len(idxCheckBoundaries) > 1:
                print("error: more than one possible solution")
            else:
                tableInit[x, y] = setV[idxCheckBoundaries[0]]
                tableArr[x, y] = lookUpFunc(setV[idxCheckBoundaries[0]])
    else:
        matDetermWrongV = np.full_like(tableInit, False)
        matDetermRightV = np.full_like(tableInit, False)
    
    
    matDetermRight = np.logical_or(matDetermRight, matDetermRightV)
    matDetermWrong = np.logical_or(matDetermWrong, matDetermWrongV)
    
    
    ##! building the rest of the external layer which now is deterministic
    
    ##! Auxiliar Important Functions
    def getAdjacentPoints(x, y, maskFrameCheck):
        
        checkFrame2 = np.full_like(tableInit, False)
        xMod = x-1
        yMod = y-1
        if xMod < 0:
            xMod = 0
        if yMod < 0:
            yMod = 0
        checkFrame2[xMod:x+2, y] = True
        checkFrame2[x, yMod:y+2] = True
        
        checkFrame2 = np.logical_and(checkFrame2, maskFrameCheck)
        return np.where(checkFrame2)
    
    
    def strategyOne(x2, y2, maskFrameCheck, boolLayerExt=False, boolPrint=False):
        
        idx2 = getAdjacentPoints(x2, y2, maskFrameCheck)
        detAdjPoints = np.array([getPointsSquare(x, y, tableArr[x,y]) for x, y in zip(*idx2)], dtype=npType)
        
        setF = np.array(['FC', 'FB', 'FE', 'FD'])
        setB = np.array(['BC', 'BB', 'BE', 'BD'])
        setV = np.array(['VC', 'VB', 'VE', 'VD'])
        setL = np.array(['LH', 'LV'])
        
        idPoint = tableInit[x2, y2]
    
        ## get the points of the boundary
        pointsXY = getAllPointsSquare(x2, y2)
        
        ## get the points and the coordinates of the deterministic external points
        matches = np.array([np.any((detAdjPointsAux[:, np.newaxis, :] == pointsXY).all(axis=2), axis=1) for detAdjPointsAux in detAdjPoints])
        idxMatch = np.where(matches)
        idxMatchMat = np.array(list(zip(*idxMatch)))
        
        ## get the proper Match points on the boundary
        if idxMatchMat.size != 0:
            properPointsBoundary = detAdjPoints[idxMatchMat[:, 0], idxMatchMat[:, 1]]
        else:
            properPointsBoundary = np.array([], dtype=npType)
        
        
        ##! hipothessis of rejecting pieces that are not possible
        pointsAdjXY = np.array([getAllPointsSquare(x, y) for x, y in zip(*idx2)], dtype=npType)
        matchesAllPossible = np.array([np.any((pointsAdjXYAux[:, np.newaxis, :] == pointsXY).all(axis=2), axis=1) for pointsAdjXYAux in pointsAdjXY])
        idxMatchAllPossible = np.where(matchesAllPossible)
        idxMatchMatAllPossible = np.array(list(zip(*idxMatchAllPossible)))
        
        maskNotPossible = np.logical_and(matchesAllPossible, np.logical_not(matches))
        idxNotPossible = np.where(maskNotPossible)
        idxMatchMatNotPossible = np.array(list(zip(*idxNotPossible)))
        
        if idxMatchMatNotPossible.size != 0:
            properPointsBoundaryNot = pointsAdjXY[idxMatchMatNotPossible[:, 0], idxMatchMatNotPossible[:, 1]]
        else:
            properPointsBoundaryNot = np.array([], dtype=npType)
        
        actions = []
        
        if idPoint in setF:
            
            ## get the points of all the possible points available
            gridPointsSet = np.array([getPointsSquare(x2, y2, lookUpFunc(x)) for x in setF], dtype=npType)
            
            if properPointsBoundaryNot.size !=0:
                checkSimilarityNot = np.array([np.any(np.all(gridPointsSetAux[:, np.newaxis, :] == properPointsBoundaryNot, axis=(2)), axis=1) for gridPointsSetAux in gridPointsSet], dtype=npType)
                checkSimilarityNot = np.sum(checkSimilarityNot, axis=1) != 0
            else:
                checkSimilarityNot = np.full((gridPointsSet.shape[0],), False, dtype=np.bool_)
            
            idxCheckBoundaries = np.full((gridPointsSet.shape[0],), True, dtype=np.bool_)
            
            ## in case of not having points on the boundary, the full set is returned and we arent in the external layer
            if properPointsBoundary.size == 0 and np.logical_not(boolLayerExt):
                return setF[np.logical_not(checkSimilarityNot)]
            
            elif properPointsBoundary.size == 0 and boolLayerExt:
                
                checkBoundaries = np.any(np.logical_or(gridPointsSet == 0, gridPointsSet == 2*mSize), axis=(1,2))
                idxCheckBoundaries = np.logical_not(np.logical_or(checkBoundaries, checkSimilarityNot))
                
                return setF[idxCheckBoundaries]
            elif boolLayerExt:
                
                checkBoundaries = np.any(np.logical_or(gridPointsSet == 0, gridPointsSet == 2*mSize), axis=(1,2))
                idxCheckBoundaries = np.logical_not(np.logical_or(checkBoundaries, checkSimilarityNot))
            
            else:
                pass
            
            ## compute the similarity between the external points and the boundary points
            similarityVec = np.array([np.any(np.all(gridPointsSetAux[:, np.newaxis, :] == properPointsBoundary, axis=(2)), axis=1) for gridPointsSetAux in gridPointsSet], dtype=npType)
            similarityVec = np.sum(similarityVec, axis=1)
        
            ## find the best match(es) : all the options with less than the maximum similarity are discarded
            actions = setF[*np.where(np.logical_and(similarityVec == np.max(similarityVec), idxCheckBoundaries))]
            
        elif idPoint in setV:
            
            ## get the points of all the possible points available
            gridPointsSet = np.array([getPointsSquare(x2, y2, lookUpFunc(x)) for x in setV], dtype=npType)
            
            if properPointsBoundaryNot.size !=0:
                checkSimilarityNot = np.array([np.any(np.all(gridPointsSetAux[:, np.newaxis, :] == properPointsBoundaryNot, axis=(2)), axis=1) for gridPointsSetAux in gridPointsSet], dtype=npType)
                checkSimilarityNot = np.sum(checkSimilarityNot, axis=1) != 0
            else:
                checkSimilarityNot = np.full((gridPointsSet.shape[0],), False, dtype=np.bool_)
            
            idxCheckBoundaries = np.full((gridPointsSet.shape[0],), True, dtype=np.bool_)
            
            ## in case of not having points on the boundary, the full set is returned and we arent in the external layer
            if properPointsBoundary.size == 0 and np.logical_not(boolLayerExt):
                return setV[np.logical_not(checkSimilarityNot)]
            
            elif properPointsBoundary.size == 0 and boolLayerExt:

                checkBoundaries = np.any(np.logical_or(gridPointsSet == 0, gridPointsSet == 2*mSize), axis=(1,2))
                idxCheckBoundaries = np.logical_not(np.logical_or(checkBoundaries, checkSimilarityNot))
                
                return setV[idxCheckBoundaries]
            elif boolLayerExt:
                
                checkBoundaries = np.any(np.logical_or(gridPointsSet == 0, gridPointsSet == 2*mSize), axis=(1,2))
                idxCheckBoundaries = np.logical_not(np.logical_or(checkBoundaries, checkSimilarityNot))
                
            else:
                pass
            
            ## compute the similarity between the external points and the boundary points
            similarityVec = np.array([np.any(np.all(gridPointsSetAux[:, np.newaxis, :] == properPointsBoundary, axis=(2)), axis=1) for gridPointsSetAux in gridPointsSet], dtype=npType)
            similarityVec = np.sum(similarityVec, axis=1)
            
            # ## find the best match(es) : all the options with less than the maximum similarity are discarded
            actions = setV[*np.where(np.logical_and(similarityVec == np.max(similarityVec), idxCheckBoundaries))]
            
        elif idPoint in setL:
            
            if properPointsBoundary.size == 0 and properPointsBoundaryNot.size == 0:
                return setL
            
            # ## get the points of all the possible points available
            gridPointsSet = np.array([getPointsSquare(x2, y2, lookUpFunc(x)) for x in setL], dtype=npType)

            if properPointsBoundaryNot.size !=0:
                checkSimilarityNot = np.array([np.any(np.all(gridPointsSetAux[:, np.newaxis, :] == properPointsBoundaryNot, axis=(2)), axis=1) for gridPointsSetAux in gridPointsSet], dtype=npType)
                checkSimilarityNot = np.sum(checkSimilarityNot, axis=1) != 0
            else:
                checkSimilarityNot = np.full((gridPointsSet.shape[0],), False, dtype=np.bool_)
            
            checkSimilarityNot = np.logical_not(checkSimilarityNot)
            
            ## compute the similarity between the external points and the boundary points
            if properPointsBoundary.size != 0:
                similarityVec = np.array([np.any(np.all(gridPointsSetAux[:, np.newaxis, :] == properPointsBoundary, axis=(2)), axis=1) for gridPointsSetAux in gridPointsSet], dtype=npType)
                similarityVec = np.sum(similarityVec, axis=1)
            else:
                similarityVec = np.zeros((gridPointsSet.shape[0],), dtype=npType)
                
            # ## find the best match(es) : all the options with less than the maximum similarity are discarded
            actions = setL[*np.where(np.logical_and(similarityVec == np.max(similarityVec), checkSimilarityNot))]
            
        elif idPoint in setB:
            
            print("idPoint: ", idPoint)
            print("properPointsBoundary: ", properPointsBoundary)
            print("properPointsBoundaryNot: ", properPointsBoundaryNot)
            
            ## in case of not having points on the boundary, the full set is returned
            if properPointsBoundary.size == 0 and properPointsBoundaryNot.size == 0:
                return setB

            # ## get the points of all the possible points available
            gridPointsSet = np.array([getPointsSquare(x2, y2, lookUpFunc(x)) for x in setB], dtype=npType)

            if properPointsBoundaryNot.size !=0:
                checkSimilarityNot = np.array([np.any(np.all(gridPointsSetAux[:, np.newaxis, :] == properPointsBoundaryNot, axis=(2)), axis=1) for gridPointsSetAux in gridPointsSet], dtype=npType)
                checkSimilarityNot = np.sum(checkSimilarityNot, axis=1) != 0
            else:
                checkSimilarityNot = np.full((gridPointsSet.shape[0],), False, dtype=np.bool_)
            
            checkSimilarityNot = np.logical_not(checkSimilarityNot)
            
            ## compute the similarity between the external points and the boundary points
            if properPointsBoundary.size != 0:
                similarityVec = np.array([np.any(np.all(gridPointsSetAux[:, np.newaxis, :] == properPointsBoundary, axis=(2)), axis=1) for gridPointsSetAux in gridPointsSet], dtype=npType)
                similarityVec = np.sum(similarityVec, axis=1)
            else:
                similarityVec = np.zeros((gridPointsSet.shape[0],), dtype=npType)
            
            # ## find the best match(es) : all the options with less than the maximum similarity are discarded
            actions = setB[*np.where(np.logical_and(similarityVec == np.max(similarityVec), checkSimilarityNot))]
    
        return actions
    
    
    ##! Build the zeroth mask Frame Check
    ##! the maskFrameCheck is the mask that tells us which points are already determined
    matDetermBoth = np.logical_or(matDetermRight, matDetermWrong)
    idxDetermBoth = np.where(matDetermBoth)
    
    matActions = np.zeros((mSize, mSize), dtype=object)
    boolNonDetermSystem = False
    
    maskFrameCheck = np.full_like(tableInit, False)
    
    
    ##! Zero Iteration
    maskAux = np.full_like(tableInit, False)
    for x, y in zip(*idxDetermBoth):
        maskFrameCheck[x, y] = True
        xMod = x-1
        yMod = y-1
        if xMod < 0:
            xMod = 0
        if yMod < 0:
            yMod = 0
            
        maskAux[xMod:x+2, y] = True
        maskAux[x, yMod:y+2] = True
        
    for x, y in zip(*idxDetermBoth):
        maskAux[x, y] = False
    
    maskBorderZero = np.logical_and(maskBorder, maskAux)
    
    idx2ChangeBorderZero = np.array(list(zip(*np.where(maskBorderZero))))
    idx2FrameCheck = np.array(list(zip(*np.where(maskFrameCheck))))
    
    
    for i in range(idx2ChangeBorderZero.shape[0]):
        x2, y2 = idx2ChangeBorderZero[i]
        actions = strategyOne(x2, y2, maskFrameCheck)
        
        if len(actions) == 1:
            tableInit[x2, y2] = actions[0]
            tableArr[x2, y2] = lookUpFunc(actions[0])
            maskFrameCheck[x2, y2] = True
        else:
            boolNonDetermSystem = True
            matActions[x2, y2] = actions
    
    
    ##! Recurrent Iterations
    boolLayerExt = np.logical_not(np.all(np.logical_and(maskFrameCheck, maskBorder)))
    maskLoopCheck = maskFrameCheck.copy()
    
    while boolLayerExt:
        
        idxDeterm = np.where(maskLoopCheck)
        
        ##! Define the mask for the new boundaries
        maskAux = np.full_like(tableInit, False)
        for x, y in zip(*idxDeterm):
            maskAux[x-1:x+2, y] = True
            maskAux[x, y-1:y+2] = True
        
        for x, y in zip(*idxDeterm):
            maskAux[x, y] = False
        
        ##! ensure we do not leave the external layer
        maskBorderZero = np.logical_and(maskBorder, maskAux)
        
        idx2ChangeBorderZero = np.array(list(zip(*np.where(maskBorderZero))))
        
        ##! check if the iteration over the external layer is finished
        if idx2ChangeBorderZero.size == 0:
            boolLayerExt = False
        
        for i in range(idx2ChangeBorderZero.shape[0]):
            x2, y2 = idx2ChangeBorderZero[i]
            actions = strategyOne(x2, y2, maskFrameCheck, boolLayerExt)
        
            if len(actions) == 1:
                tableInit[x2, y2] = actions[0]
                tableArr[x2, y2] = lookUpFunc(actions[0])
                maskFrameCheck[x2, y2] = True
                matActions[x2, y2] = 0
            else:
                matActions[x2, y2] = actions
                
            maskLoopCheck[x2, y2] = True
    
    def findBetterPoint(maskFrame, maskFrameCheck):
        x2 = 0
        y2 = 0
        counterMax = 0
        idxMaskFrame = np.where(maskFrame)
        for x, y in zip(*idxMaskFrame):
            maskAuxUpper = np.full_like(tableInit, False)
            xMin = x-1
            yMax = y-1
            xMax = x+1
            yMin = y+1
            if xMin < 0:
                xMin = 0
            if yMin < 0:
                yMin = 0
            if xMax >= mSize:
                xMax = mSize-1
            if yMin >= mSize:
                yMin = mSize-1
            maskAuxUpper[xMin, y] = True
            maskAuxUpper[x, yMax] = True
            maskAuxUpper[xMax, y] = True
            maskAuxUpper[x, yMin] = True
            
            maskResult = np.logical_and(maskAuxUpper, maskFrameCheck)
            
            # print("mask Result: ", maskResult)
            # print("*"*30)
            
            counter = np.sum(maskResult)
            
            if counter > counterMax:
                counterMax = counter
                x2 = x
                y2 = y
                
        return x2, y2, counterMax
    
    
    ##! Deterministic inference Upper Layer
    
    def strategyTwoLoop(loopNumber):
        
        maskLevelUpper = np.full_like(tableInit, False)
        maskLevelUpper[loopNumber:-loopNumber, loopNumber] = True
        maskLevelUpper[loopNumber:-loopNumber, -loopNumber-1] = True
        maskLevelUpper[loopNumber, loopNumber:-loopNumber] = True
        maskLevelUpper[-loopNumber-1, loopNumber:-loopNumber] = True
        
        boolUpperLayer = np.any(np.logical_and(maskLevelUpper, np.logical_not(maskFrameCheck)))
        
        ##! Fill the upper layer: it ends up when there are no more deterministic points to fill
        while boolUpperLayer:
            x2, y2, counterMax = findBetterPoint(maskLevelUpper, maskFrameCheck)
            
            if counterMax == 0:
                boolUpperLayer = False
            else:
                actions = strategyOne(x2, y2, maskFrameCheck, False)
                
                if len(actions) == 1:
                    tableInit[x2, y2] = actions[0]
                    tableArr[x2, y2] = lookUpFunc(actions[0])
                    maskFrameCheck[x2, y2] = True
                    matActions[x2, y2] = 0
                else:
                    matActions[x2, y2] = actions
                
                maskLevelUpper[x2, y2] = False
        
        
        maskLevelUpper = np.full_like(tableInit, False)
        maskLevelUpper[loopNumber:-loopNumber, loopNumber] = True
        maskLevelUpper[loopNumber:-loopNumber, -loopNumber-1] = True
        maskLevelUpper[loopNumber, loopNumber:-loopNumber] = True
        maskLevelUpper[-loopNumber-1, loopNumber:-loopNumber] = True
        
        maskFrameCheck2nd = np.logical_and(np.logical_not(maskFrameCheck), maskLevelUpper)
        
        for x2, y2 in zip(*np.where(maskFrameCheck2nd)):
            actions = strategyOne(x2, y2, maskFrameCheck)
            
            if len(actions) == 1:
                tableInit[x2, y2] = actions[0]
                tableArr[x2, y2] = lookUpFunc(actions[0])
                maskFrameCheck[x2, y2] = True
                matActions[x2, y2] = 0
            else:
                matActions[x2, y2] = actions


    if mSize >= 3:
        mLoops = int(np.ceil(mSize/2))
        
        for i in range(1, mLoops):
            strategyTwoLoop(i)
        
        ##! Final Deterministic Inference where the actions can be reduced in unsolved domain
        maskFrameCheck3rd = np.logical_not(maskFrameCheck)
        bool3rdLayer = np.any(maskFrameCheck3rd)
        
        while bool3rdLayer:
            x2, y2, counterMax = findBetterPoint(maskFrameCheck3rd, maskFrameCheck)
            
            if counterMax == 0:
                bool3rdLayer = False
            else:
                boolLayerExt = False
                if x2 == 0 or x2 == mSize-1 or y2 == 0 or y2 == mSize-1:
                    boolLayerExt = True
                actions = strategyOne(x2, y2, maskFrameCheck, boolLayerExt)
                
                if len(actions) == 1:
                    tableInit[x2, y2] = actions[0]
                    tableArr[x2, y2] = lookUpFunc(actions[0])
                    maskFrameCheck[x2, y2] = True
                    matActions[x2, y2] = 0
                else:
                    matActions[x2, y2] = actions
                
                maskFrameCheck3rd[x2, y2] = False
    else:
        pass

    
    
    print("Final table: \n", tableInit)
    print("Actions: \n", matActions)
    print("Frame Check: \n", maskFrameCheck)



##! Test cases

## test-01.txt
tableInit2 = np.array(
    [['VC', 'VD'] \
    ,['FB', 'FB']], dtype=object)

## test-05.txt
tableInit3 = np.array(
    [['FC', 'FE', 'VC'] \
    ,['BC', 'LV', 'BD'] \
    ,['VE', 'FB', 'FB']] , dtype=object)


## test-07.txt
tableInit4 = np.array(  
    [['FC','BB','BC','FB'],
     ['FC','BD','FD','FD'],
     ['VD','BC','BC','BD'],
     ['FB','FB','VE','FD']] , dtype=object)


## test-08.txt
tableInit5 = np.array(
    [['VE','FD','VE','LH','FB'],
     ['LV','FC','BB','LH','VE'],
     ['BE','VB','BC','FC','LV'],
     ['VE','BD','BD','VD','LV'],
     ['FC','VE','FE','VE','FD']] , dtype=object)


## test-09.txt
tableInit6 = np.array(
    [['FE','VD','FE','VE','VB'],
     ['VE','BC','LV','FE','LV'],
     ['FE','BB','BC','BC','BE'],
     ['FE','VD','FD','BD','LV'],
     ['FC','LH','LH','VE','FD']] , dtype=object)


import time

print("Test Cases")
print()

initTime = time.time()

# print("Initial table: ", tableInit2)
# deterministicInference(tableInit2)
# print()
# print("Initial table: ", tableInit3)
# deterministicInference(tableInit3)
# print()
print("Initial table: \n", tableInit5)
deterministicInference(tableInit5)

finalTime = time.time()

print()
print("Time: ", finalTime - initTime)


Test Cases

Initial table: 
 [['VE' 'FD' 'VE' 'LH' 'FB']
 ['LV' 'FC' 'BB' 'LH' 'VE']
 ['BE' 'VB' 'BC' 'FC' 'LV']
 ['VE' 'BD' 'BD' 'VD' 'LV']
 ['FC' 'VE' 'FE' 'VE' 'FD']]
idPoint:  BB
properPointsBoundary:  [[2 5]
 [3 6]]
properPointsBoundaryNot:  []
idPoint:  BD
properPointsBoundary:  [[7 2]
 [8 3]]
properPointsBoundaryNot:  []
idPoint:  BD
properPointsBoundary:  []
properPointsBoundaryNot:  [[8 5]]
idPoint:  BB
properPointsBoundary:  [[2 5]
 [3 6]]
properPointsBoundaryNot:  []
idPoint:  BD
properPointsBoundary:  [[7 2]
 [7 4]
 [8 3]]
properPointsBoundaryNot:  []
idPoint:  BC
properPointsBoundary:  [[6 5]]
properPointsBoundaryNot:  []
idPoint:  BC
properPointsBoundary:  [[6 5]]
properPointsBoundaryNot:  []
idPoint:  BB
properPointsBoundary:  [[2 5]
 [3 6]]
properPointsBoundaryNot:  []
idPoint:  BC
properPointsBoundary:  [[5 6]
 [6 5]]
properPointsBoundaryNot:  []
Final table: 
 [['VB' 'FE' 'VB' 'LH' 'FE']
 ['LV' 'FC' 'BB' 'LH' 'VE']
 ['BD' 'VB' 'BC' 'FE' 'LV']
 ['VD' 'BB' 'BC' 'VE' 'LV

In [8]:
import numpy as np

def update_adjacent_multiple(matrix, entries):
    max_rows, max_cols = matrix.shape

    # Create arrays for row and column indices for each entry
    row_indices = np.array(entries)[:, 0]
    col_indices = np.array(entries)[:, 1]

    # Define relative positions for adjacent cells
    relative_positions = np.array([[-1, 0], [1, 0], [0, -1], [0, 1]])

    # Compute all adjacent positions
    all_positions = row_indices[:, None] + relative_positions[:, 0], col_indices[:, None] + relative_positions[:, 1]

    # Filter positions that are inside the matrix bounds
    valid = (all_positions[0] >= 0) & (all_positions[0] < max_rows) & \
            (all_positions[1] >= 0) & (all_positions[1] < max_cols)

    # Flatten valid positions and make them unique to avoid repeated setting of the same cell
    valid_rows = all_positions[0][valid]
    valid_cols = all_positions[1][valid]
    unique_positions = np.unique((valid_rows, valid_cols), axis=1)

    # Set the valid adjacent positions to True
    matrix[unique_positions[0], unique_positions[1]] = True

# Example matrix
matrix = np.full((5, 5), False, dtype=bool)

# Positions of the entries (e.g., multiple central points in a larger matrix)
entries = [(1, 1), (3, 3), (1, 3)]

# Updating the adjacent entries of the specified entries
update_adjacent_multiple(matrix, entries)

# Print the updated matrix
print(matrix)


[[False  True False  True False]
 [ True False  True False  True]
 [False  True False  True False]
 [False False  True False  True]
 [False False False  True False]]


In [10]:
list(zip(*np.where(matrix)))

[(0, 1),
 (0, 3),
 (1, 0),
 (1, 2),
 (1, 4),
 (2, 1),
 (2, 3),
 (3, 2),
 (3, 4),
 (4, 3)]